In [1]:
import pandas as pd
import numpy as np
import subprocess
import re
import scipy as sp

# Set input variables
gwas_file_path = "/Users/hn9/Documents/Analysis/FM-comparison/gwas-examples/APOE-LDL/24097068-GCST002222-EFO_0004611.h.tsv.gz"
target = "APOE_LDL"
target_chrom = 19
target_pos = 44908822
start_pos = target_pos - 500000
end_pos = target_pos + 500000
lead_snp_ID = f"{target_chrom}:{target_pos}:C:T"
n_sample = 94595

In [ ]:
def get_ld_matrix():
    # Calculate LD correlation
    ld_data = pd.read_csv(f"{target}_locus_UKBB.txt.raw", delim_whitespace=True)
    ld_data = ld_data.drop(columns=["FID", "IID", "PAT", "MAT", "SEX", "PHENOTYPE"])
    ld_matrix = pd.DataFrame(np.corrcoef(df.values, rowvar=False), columns=df.columns)
    return ld_data, ld_matrix


In [2]:
def get_ld_matrix():
    # Calculate LD correlation
    ld_data = pd.read_csv(f"{target}_locus_UKBB.txt.raw", delim_whitespace=True)
    ld_data = ld_data.drop(columns=["FID", "IID", "PAT", "MAT", "SEX", "PHENOTYPE"])
    ld_matrix = ld_data.corr(method='pearson')
    return ld_data, ld_matrix


def get_sumstats(gwas_file_path, target_chrom, start_pos, end_pos):
    # Preparing sumstats for filtering and input into SuSiE
    sumstat = pd.read_csv(gwas_file_path, sep="\t")
    sumstat = sumstat[(sumstat['hm_chrom'] == target_chrom) & (sumstat['hm_pos'] >= start_pos) & (sumstat['hm_pos'] <= end_pos)]
    sumstat = sumstat.dropna(subset=['hm_chrom'])
    sumstat['z'] = sumstat['beta'] / sumstat['standard_error']
    cols_to_rename = {
        'hm_variant_id': 'ID',
        'hm_rsid': 'rsid',
        'hm_chrom': 'chromosome',
        'hm_pos': 'position',
        'hm_other_allele': 'allele1',
        'hm_effect_allele': 'allele2',
        'hm_effect_allele_frequency': 'maf',
        'standard_error': 'se',
        'p_value': 'p'
    }
    sumstat.rename(columns=cols_to_rename, inplace=True)
    sumstat = sumstat[['ID', 'rsid', 'chromosome', 'position', 'allele1', 'allele2', 'maf', 'p', 'beta', 'se', 'z']]
    return sumstat


def match_snps(sumstat, ld_matrix, ld_data):
    # Getting only SNPs in sumstats that are in the LD matrix
    # Get SNP IDs from ld matrix to compare with sumstat ID
    pattern = re.compile(r"(^\d+)|(?<=:)\d+(?=:|$)")
    df1_transpose = ld_data.T.reset_index()
    df1_transpose.columns = ['SNP'] + list(df1_transpose.columns[1:])
    df1_transpose['position'] = df1_transpose['SNP'].apply(lambda x: re.search(pattern, x).group())
    df1_transpose['ID'] = df1_transpose['SNP'].str.replace(r'[:,_]', '_').str.replace(r'_[^_]+$', '')
    concordance_test = pd.merge(sumstat, df1_transpose, on='ID')

    # Filter sumstat and LD matrix for matches only
    sumstat_filtered = sumstat[sumstat['ID'].isin(concordance_test['ID'])]
    sumstat_filtered.reset_index(drop=True, inplace=True)
    ld_matrix_filtered = ld_matrix.loc[concordance_test['SNP'], concordance_test['SNP']]
    return sumstat_filtered, ld_matrix_filtered, concordance_test


def allele_flip_check(concordance_test, sumstat_filtered, ld_matrix_filtered):
    # Create DataFrame for allele flip check
    concordance_test2 = concordance_test.copy()

    # Extract alleles
    allele_df = concordance_test2['SNP'].str.extract(r'[:,_]([ACGT]+)[:,_]([ACGT]+)')
    concordance_test2['allele1_LD'] = allele_df[0]
    concordance_test2['allele2_LD'] = allele_df[1]

    # Make sure indices are aligned before the next operation
    sumstat_filtered.index = concordance_test2.index

    # Flip z-scores if alleles are discordant
    sumstat_filtered['z'] = np.where(
        (sumstat_filtered['allele1'] != concordance_test2['allele1_LD']) | (sumstat_filtered['allele2'] != concordance_test2['allele2_LD']),
        -sumstat_filtered['z'], sumstat_filtered['z']
    )

    # Check for NaNs or Inf values (not accepted in fine-mapping)
    numeric_cols = sumstat_filtered.select_dtypes(include=[np.number])
    any_na_matrix = ld_matrix_filtered.isna().any().any()
    any_inf_matrix = np.isinf(ld_matrix_filtered.to_numpy()).any()
    any_na_sumstat = numeric_cols.isna().any().any()
    any_inf_sumstat = np.isinf(numeric_cols.to_numpy()).any()

    print(f"Correlation matrix has NaNs: {any_na_matrix}, Infs: {any_inf_matrix}")
    print(f"Sumstat has NaNs: {any_na_sumstat}, Infs: {any_inf_sumstat}")
    ld_matrix_filtered.to_csv(f"{target}_locus_ukbb_ld.txt.gz", sep='\t', index=False, header=False)
    sumstat_filtered.to_csv(f"{target}_locus_sumstat_flip_check.txt.gz", sep='\t', index=False)

    return sumstat_filtered


def outlier_detection(sumstat, method, r2_threshold = 0.6, nlog10p_dentist_s_threshold = 1e-4):
    if method == 'DENTIST':
        # 1. Getting R2 column for sumstats
        # LD for all variants with lead SNP previously calculated by plink
        ld = pd.read_csv(f'/Users/hn9/Documents/GitHub/fine-mapping-inf/susieinf/loci/ld/{target}_subset_for_ld_calculation.ld', sep='\s+')
        lead_ld = ld[(ld['SNP_A'] == f'{lead_snp_ID}') | (ld['SNP_B'] == f'{lead_snp_ID}')]
        sumstat = pd.read_csv(f'/Users/hn9/Documents/GitHub/fine-mapping-inf/susieinf/loci/{target}_locus_sumstat_flip_check.txt.gz', sep='\t')
        sumstat['ID'] = sumstat['ID'].str.replace("_", ":")
        merged = pd.merge(lead_ld, sumstat[['ID']], left_on='SNP_B', right_on='ID')
        df = merged[['ID', 'R2']]
        df = pd.merge(sumstat, merged, on='ID', how='left')
        df['r'] = (n_sample * df['R2'].sum()) / (n_sample * df['R2'].count())
        lead = df[df['ID'] == lead_snp_ID].iloc[0]
        lead_idx_snp = df.index[df['ID'] == lead_snp_ID].tolist()[0]

        # 2. Calculate 't_dentist_s' and 'dentist_outlier'
        # Copied and unaltered from https://github.com/mkanai/slalom/blob/master/slalom.py
        lead_z = (df.beta / df.se).iloc[lead_idx_snp]
        df["t_dentist_s"] = ((df.beta / df.se) - df.r * lead_z) ** 2 / (1 - df.r ** 2)
        df["t_dentist_s"] = np.where(df["t_dentist_s"] < 0, np.inf, df["t_dentist_s"])
        df["t_dentist_s"].iloc[lead_idx_snp] = np.nan
        df["nlog10p_dentist_s"] = sp.stats.chi2.logsf(df["t_dentist_s"], df=1) / -np.log(10)
        n_dentist_s_outlier = np.sum(
                    (df.R2 > r2_threshold) & (df.nlog10p_dentist_s > nlog10p_dentist_s_threshold)
                )
        print('Number of DENTIST outliers detected:', n_dentist_s_outlier)
        # Identifying outliers
        df['dentist_outlier'] = np.where((df.R2 > r2_threshold) & (df.nlog10p_dentist_s > nlog10p_dentist_s_threshold), 1, 0)
        df = df.drop(['CHR_A', 'BP_A', 'SNP_A', 'CHR_B', 'BP_B', 'SNP_B'], axis=1)
        df.to_csv(f'{target}_locus_sumstat_with_dentist.txt.gz', sep='\t', index=False)
    elif method == 'CARMA':
        # unfinished work in progress
        from scipy import optimize
        import carmapy.carmapy_c
        sumstats = pd.read_csv(f"{target}_locus_sumstat_flip_check.txt.gz", sep='\t')
        ld = pd.read_csv(f"{target}_locus_ukbb_ld.txt.gz", sep='\t', header=None)
        outlier_tau = 0.04
        index_list = sumstats.index.tolist()
        z = sumstats['z'].tolist()
        ld_matrix = np.asmatrix(ld)
        modi_ld_S = ld_matrix
        def ridge_fun(x, modi_ld_S, index_list, temp_Sigma, z, outlier_tau, outlier_likelihood):
            temp_ld_S = x * modi_ld_S + (1 - x) * np.eye(modi_ld_S.shape[0])
            ld_matrix[index_list[:, None], index_list] = temp_ld_S
            return outlier_likelihood(index_list, ld_matrix, z, outlier_tau, len(index_list), 1)

        opizer = optimize(ridge_fun, interval=[0, 1], maximum=True)
        modi_ld = opizer['maximum'] * modi_ld_S + (1 - opizer['maximum']) * np.diag(np.diag(modi_ld_S))
        outlier_likelihood = carmapy.carmapy_c.outlier_Normal_fixed_sigma_marginal
        test_log_BF = outlier_likelihood(index_list, ld_matrix, z, outlier_tau, len(index_list), 1) - outlier_likelihood(index_list, modi_ld, z, outlier_tau, len(index_list), 1)
        test_log_BF = -abs(test_log_BF)
        print('Outlier BF:', test_log_BF)
        print('This is xi hat:', opizer)
    else:
        pass
    return df


In [4]:
# PLINK command to get LD matrix (placeholder using PLINK and UKBiobank LD reference panel)
command = f"plink --bfile /Users/hn9/Documents/Analysis/FM-comparison/ukb_v3_downsampled10k/ukb_v3_chr{target_chrom}.downsampled10k --allow-extra-chr --recode A --chr {target_chrom} --from-bp {start_pos} --to-bp {end_pos} --maf 0.001 --out {target}_locus_UKBB.txt"
subprocess.run(command, shell=True)

# Run functions
ld_data, ld_matrix = get_ld_matrix()
sumstat = get_sumstats(gwas_file_path, target_chrom, start_pos, end_pos)
sumstat_filtered, ld_matrix_filtered, concordance_test = match_snps(sumstat, ld_matrix, ld_data)
sumstat_filtered = allele_flip_check(concordance_test, sumstat_filtered, ld_matrix_filtered)

# PLINK command to get SNPs in LD with lead SNP
lead_ld_command = f"""plink --bfile /Users/hn9/Documents/Analysis/FM-comparison/ukb_v3_downsampled10k/ukb_v3_chr{target_chrom}.downsampled10k \
        --allow-extra-chr \
        --r2 \
        --ld-snp {lead_snp_ID} \
        --ld-window-kb 1000 \
        --ld-window 99999 \
        --ld-window-r2 0 \
        --out /Users/hn9/Documents/GitHub/fine-mapping-inf/susieinf/loci/ld/{target}_subset_for_ld_calculation
"""
subprocess.run(lead_ld_command, shell=True)

# Function to calculate DENTIST outlier detection
df = outlier_detection(sumstat_filtered, method='DENTIST')


PLINK v1.90b6.21 64-bit (19 Oct 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to APOE_LDL_locus_UKBB.txt.log.
Options in effect:
  --allow-extra-chr
  --bfile /Users/hn9/Documents/Analysis/FM-comparison/ukb_v3_downsampled10k/ukb_v3_chr19.downsampled10k
  --chr 19
  --from-bp 44408822
  --maf 0.001
  --out APOE_LDL_locus_UKBB.txt
  --recode A
  --to-bp 45408822

16384 MB RAM detected; reserving 8192 MB for main workspace.
6167 out of 364540 variants loaded from .bim file.
10000 people (0 males, 0 females, 10000 ambiguous) loaded from .fam.
Ambiguous sex IDs written to APOE_LDL_locus_UKBB.txt.nosex .
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 10000 founders and 0 nonfounders present.
Calculating allele frequencies... 1011121314151617181920212223242526272829303132333435363738394041424344454647484950515253545556575859606162636465666768697071727374757677787980818

/var/folders/mp/mm_rrrpj4cj0sf0lspp7vmvm0000gs/T/ipykernel_33708/2415294170.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["t_dentist_s"].iloc[lead_idx_snp] = np.nan


Number of DENTIST outliers detected: 1


In [10]:
# SuSiE fine-mapping with fine-mapping-inf package
susieinf_command = f"""python /Users/hn9/Documents/GitHub/fine-mapping-inf/run_fine_mapping.py \
    --sumstats /Users/hn9/Documents/GitHub/fine-mapping-inf/susieinf/loci/{target}_locus_sumstat_with_dentist.txt.gz \
    --beta-col-name beta \
    --se-col-name se \
    --ld-file /Users/hn9/Documents/GitHub/fine-mapping-inf/susieinf/loci/{target}_locus_ukbb_ld.txt.gz \
    --n {n_sample} \
    --method susieinf \
    --save-tsv \
    --eigen-decomp-prefix {target}_locus \
    --output-prefix  {target}_locus """

subprocess.run(susieinf_command, shell=True)

zip_comamnd = f"""gunzip -c {target}_locus.susieinf.bgz > {target}_locus.txt"""
subprocess.run(zip_comamnd, shell=True)

Traceback (most recent call last):
  File "/Users/hn9/Documents/GitHub/fine-mapping-inf/run_fine_mapping.py", line 7, in <module>
    import bgzip
  File "/Users/hn9/anaconda3/envs/finemap_env/lib/python3.9/site-packages/bgzip/__init__.py", line 7, in <module>
    from bgzip import bgzip_utils  # type: ignore
ImportError: dlopen(/Users/hn9/anaconda3/envs/finemap_env/lib/python3.9/site-packages/bgzip/bgzip_utils.cpython-39-darwin.so, 0x0002): symbol not found in flat namespace '___kmpc_barrier'
gunzip: can't stat: APOE_LDL_locus.susieinf.bgz (APOE_LDL_locus.susieinf.bgz.gz): No such file or directory


CompletedProcess(args='gunzip -c APOE_LDL_locus.susieinf.bgz > APOE_LDL_locus.txt', returncode=1)

# DENTIST Test Code

In [25]:
# 1. Getting R2 column for sumstats
# LD for all variants with lead SNP previously calculated by plink
ld = pd.read_csv(f'/Users/hn9/Documents/GitHub/fine-mapping-inf/susieinf/loci/ld/{target}_subset_for_ld_calculation.ld', sep='\s+')
lead_ld = ld[(ld['SNP_A'] == f'{lead_snp_ID}') | (ld['SNP_B'] == f'{lead_snp_ID}')]
sumstat = pd.read_csv(f'/Users/hn9/Documents/GitHub/fine-mapping-inf/susieinf/loci/{target}_locus_sumstat_flip_check.txt.gz', sep='\t')
sumstat['ID'] = sumstat['ID'].str.replace("_", ":")
merged = pd.merge(lead_ld, sumstat[['ID']], left_on='SNP_B', right_on='ID')
df = merged[['ID', 'R2']]
df = pd.merge(sumstat, merged, on='ID', how='left')
df['r'] = (n_sample * df['R2'].sum()) / (n_sample * df['R2'].count())
lead = df[df['ID'] == lead_snp_ID].iloc[0]
lead_idx_snp = df.index[df['ID'] == lead_snp_ID].tolist()[0]

# 2. Calculate 't_dentist_s' and 'dentist_outlier'
# Copied and unaltered from https://github.com/mkanai/slalom/blob/master/slalom.py
lead_z = (df.beta / df.se).iloc[lead_idx_snp]
df["t_dentist_s"] = ((df.beta / df.se) - df.r * lead_z) ** 2 / (1 - df.r ** 2)
df["t_dentist_s"] = np.where(df["t_dentist_s"] < 0, np.inf, df["t_dentist_s"])
df["t_dentist_s"].iloc[lead_idx_snp] = np.nan
df["nlog10p_dentist_s"] = sp.stats.chi2.logsf(df["t_dentist_s"], df=1) / -np.log(10)
r2_threshold = 0.1
nlog10p_dentist_s_threshold = 1e-4
n_dentist_s_outlier = np.sum(
            (df.R2 > r2_threshold) & (df.nlog10p_dentist_s > nlog10p_dentist_s_threshold)
        )

# Identifying outliers
df['dentist_outlier'] = np.where((df.R2 > r2_threshold) & (df.nlog10p_dentist_s > nlog10p_dentist_s_threshold), 1, 0)
df = df.drop(['CHR_A', 'BP_A', 'SNP_A', 'CHR_B', 'BP_B', 'SNP_B'], axis=1)
#df.to_csv(f'{target}_locus_sumstat_with_dentist.txt.gz', sep='\t', index=False)

/var/folders/mp/mm_rrrpj4cj0sf0lspp7vmvm0000gs/T/ipykernel_12730/1863148979.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["t_dentist_s"].iloc[lead_idx_snp] = np.nan


In [27]:
total = df['dentist_outlier'].sum()
print('sum',n_dentist_s_outlier)
print(df[df['dentist_outlier'] == 1]
)

sum 10
                  ID        rsid  chromosome  position allele1 allele2  \
169  19:44738916:G:A   rs1531517          19  44738916       G       A   
170  19:44744370:A:G   rs4803750          19  44744370       A       G   
214  19:44870308:G:A    rs395908          19  44870308       G       A   
226  19:44885917:T:A    rs283813          19  44885917       T       A   
227  19:44886339:G:A   rs7254892          19  44886339       G       A   
238  19:44897490:T:A  rs61679753          19  44897490       T       A   
256  19:44910319:C:T  rs75627662          19  44910319       C       T   
258  19:44912383:G:A    rs445925          19  44912383       G       A   
268  19:44929300:G:C   rs7259004          19  44929300       G       C   
273  19:44943964:G:A  rs12721109          19  44943964       G       A   

         maf              p    beta      se          z        R2         r  \
169  0.94855  9.510000e-163  0.2202  0.0080  27.525000  0.166354  0.014799   
170  0.94459  1.700000

# Testing Dentist Calculation on SLALOM data

In [5]:

df = pd.read_csv('/Users/hn9/Documents/GitHub/slalom-susie/python-pipeline-tests/example.slalom.txt', sep='\t')


In [7]:
df.columns

Index(['rsid', 'chromosome', 'position', 'allele1', 'allele2', 'maf', 'beta',
       'se', 'p', 'all_meta_AF', 'p_het', 'n_cases', 'n_controls', 'n_samples',
       'n_datasets', 'n_biobanks', 'is_strand_flip', 'is_diff_AF_gnomAD',
       'n_afr', 'n_amr', 'n_eas', 'n_fin', 'n_nfe', 'in_cups', 'most_severe',
       'gene_most_severe', 'consequence', 'gnomad_v3_af_afr',
       'gnomad_v3_af_amr', 'gnomad_v3_af_eas', 'gnomad_v3_af_fin',
       'gnomad_v3_af_nfe', 'gnomad_v3_af_sas', 'lbf', 'prob', 'cs', 'cs_99',
       'lead_variant', 'gnomad_lead_r_afr', 'gnomad_lead_r_amr',
       'gnomad_lead_r_eas', 'gnomad_lead_r_fin', 'gnomad_lead_r_nfe', 'r',
       't_dentist_s', 'nlog10p_dentist_s'],
      dtype='object')

In [8]:
df['t_dentist_s']

0       11.892953
1        0.276961
2        1.893043
3             NaN
4       14.072130
          ...    
1088     1.598529
1089          NaN
1090          NaN
1091          NaN
1092          NaN
Name: t_dentist_s, Length: 1093, dtype: float64

In [12]:
df['z'] = df['beta']/df['se']
lead_z = df[(df['rsid'] == 'rs2099684')]
lead_z_value = lead_z['z'].iloc[0]
df['t_dentist_s2'] = (df['z'] - df['r'] * lead_z_value)**2 / (1 - df['r']**2)
df['t_dentist_s2']

0       11.892953
1        0.276961
2        1.893043
3             NaN
4       14.072130
          ...    
1088     1.598529
1089          NaN
1090          NaN
1091          NaN
1092          NaN
Name: t_dentist_s2, Length: 1093, dtype: float64

# CARMA Outlier detection Test code

In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import carmapy.carmapy_c

sumstats = pd.read_csv("/Users/hn9/Documents/GitHub/fine-mapping-inf/susieinf/loci/APOE_LDL_locus_sumstat_with_dentist.txt.gz", sep='\t')
ld = pd.read_csv("/Users/hn9/Documents/GitHub/fine-mapping-inf/susieinf/loci/APOE_LDL_locus_ukbb_ld.txt.gz", sep='\t', header=None)
outlier_tau = 0.04

index_list = np.array(sumstats.index.tolist()).astype(np.uint32)
z = np.array(sumstats['z'].tolist()).astype(np.float64)
ld_matrix = np.array(ld, dtype=np.float64)
modi_ld_S = ld_matrix

outlier_likelihood = carmapy.carmapy_c.outlier_Normal_fixed_sigma_marginal  # Replace with the actual function if different

def ridge_fun(x, modi_ld_S, index_list, ld_matrix, z, outlier_tau, outlier_likelihood):
    x_scalar = x[0]
    temp_ld_S = x_scalar * modi_ld_S + (1 - x_scalar) * np.eye(modi_ld_S.shape[0])
    ld_matrix[index_list[:, None], index_list] = temp_ld_S
    return outlier_likelihood(index_list, ld_matrix, z, outlier_tau, len(index_list), 1)

# Optimization
initial_guess = np.array([0.5])  # Initial guess as an array
result = minimize(
    lambda x: -ridge_fun(x, modi_ld_S, index_list, ld_matrix, z, outlier_tau, outlier_likelihood),
    initial_guess,  # Use the initial guess array here
    bounds=[(0, 1)]
)

# Extract optimized parameter
optimized_x = result.x[0]  # Extract the scalar value from the array

# Modify LD matrix
modi_ld = optimized_x * modi_ld_S + (1 - optimized_x) * np.diag(np.diag(modi_ld_S))

# Calculate test_log_BF
test_log_BF = outlier_likelihood(index_list, ld_matrix, z, outlier_tau, len(index_list), 1) - \
              outlier_likelihood(index_list, modi_ld, z, outlier_tau, len(index_list), 1)
test_log_BF = -abs(test_log_BF)

print('Outlier BF:', test_log_BF)
print('This is xi hat:', optimized_x)
